In [1]:
import pandas as pd
import numpy as np
import os
import requests

In [2]:
import warnings
warnings.filterwarnings('ignore')

# Concatenate monthly-end Billboard Hot 100

In [9]:
path = 'billboard_data/'
file = os.listdir(path)

year = [i for i in range(2012, 2022)]

file_name = sorted([f for f in file if f[:4] in str(year)])

hit = pd.read_csv(path_b + file_name[0])

for f in file_name[1:]:
    hit = pd.concat([hit,
                     pd.read_csv(path + f)],
                     ignore_index=True)
    
# add chart date for further reference 
## (i.e. if we want to denote songs ever featured top 20 as hit, chart date matters)
date = []

for i in file_name:
    date += [i[:-4]] *100
    
hit['date'] = date

hit

,artist,isNew,last,peak,rank,title,week,date
0,Adele,False,2,1,1,Set Fire To The Rain,21,2012-02-04
1,Rihanna Featuring Calvin Harris,False,1,1,2,We Found Love,18,2012-02-04
2,Flo Rida,False,3,3,3,Good Feeling,17,2012-02-04
3,David Guetta Featuring Nicki Minaj,False,10,4,4,Turn Me On,8,2012-02-04
4,Katy Perry,False,6,3,5,The One That Got Away,15,2012-02-04
...,...,...,...,...,...,...,...,...
11595,Young Thug,True,0,96,96,Tick Tock,1,2021-09-04
11596,BTS,False,66,1,97,Permission To Dance,7,2021-09-04
11597,Nio Garcia X J Balvin X Bad Bunny,False,85,41,98,AM,9,2021-09-04
11598,Rod Wave,True,0,99,99,Time Heals,1,2021-09-04


In [17]:
hit_unique = hit.drop_duplicates(subset=['title','artist'])[['title', 'artist','date']]
hit_unique['hit'] = [1]*len(hit_unique)
hit_unique

,title,artist,date,hit
0,Set Fire To The Rain,Adele,2012-02-04,1
1,We Found Love,Rihanna Featuring Calvin Harris,2012-02-04,1
2,Good Feeling,Flo Rida,2012-02-04,1
3,Turn Me On,David Guetta Featuring Nicki Minaj,2012-02-04,1
4,The One That Got Away,Katy Perry,2012-02-04,1
...,...,...,...,...
11593,Demon Time,Trippie Redd Featuring Ski Mask The Slump God,2021-09-04,1
11594,Knowing You,Kenny Chesney,2021-09-04,1
11595,Tick Tock,Young Thug,2021-09-04,1
11598,Time Heals,Rod Wave,2021-09-04,1


In [14]:
hit_unique['track_id'] = [np.nan] * len(hit_unique)
hit_unique['release_date'] = [np.nan] * len(hit_unique)
hit_unique['spotify.name'] = [np.nan] * len(hit_unique)
hit_unique['spotify.artists'] = [np.nan] * len(hit_unique)

## Get Spotify track ID

In [15]:
def getArtistNameList(result_artists):
    artists = []
    for a in result_artists:
        artists.append(a['name'])
    return artists

In [20]:
CLIENT_ID = '6ced8cf12e0c470f9d9dd25a75d4ec26'
CLIENT_SECRET = '4ce366cebd7c42f2ac59bf96ca9f32b9'

AUTH_URL = 'https://accounts.spotify.com/api/token'

# POST
auth_response = requests.post(AUTH_URL, {
    'grant_type': 'client_credentials',
    'client_id': CLIENT_ID,
    'client_secret': CLIENT_SECRET,
})

# convert the response to JSON
auth_response_data = auth_response.json()

# save the access token
access_token = auth_response_data['access_token']
headers = {
    'Authorization': 'Bearer {token}'.format(token=access_token)
}

# base URL of all Spotify API endpoints
BASE_URL = 'https://api.spotify.com/v1/'

try:
    start_idx = hit_unique.loc[~hit_unique['track_id'].isnull(),:].index.values[-1] + 1
except:
    start_idx = 0
    
for i in range(start_idx, len(hit_unique)):
    # direct search by title and artist
    query = 'track:"%s" AND artist:%s AND year:%s&type=track&limit=50'%(hit_unique['title'][i],
                                                                        hit_unique['artist'][i],
                                                                        hit_unique['date'][i][:4])

    r = requests.get(BASE_URL + 'search?q=' + query, headers=headers).json()

    try:
        results = pd.json_normalize(r['tracks']['items']).sort_values('album.release_date')

        for j in range(len(results)):
            artists_i = getArtistNameList(results['artists'][j])
            #if sum([(a in sr['artists'][i]) for a in artists_i])>=1:
            for a in artists_i:
                #if a in sr['artists'][i]:
                if (hit_unique['artists'][i].split()[0].lower() in a.split()[0].lower()) and\
                (hit_unique['artists'][i].split()[-1].lower() in a.split()[-1].lower()):
                    print(i, results['id'][j])
                    hit_unique['track_id'][i] = results['id'][j]
                    hit_unique['release_date'][i] = results['album.release_date'][j]
                    hit_unique['spotify.name'][i] = results['name'][j]
                    hit_unique['spotify.artists'][i] = artists_i
                    break
            break
    except:
        pass

KeyError: 101

In [21]:
query

'track:"Part Of Me" AND artist:Katy Perry AND year:2012&type=track&limit=50'